# Imports

In [ ]:
import datetime as dt

import matplotlib.pyplot as plt
import pymongo

# MongoDB Connection

In [ ]:
user = 'user'
password = 'pass'
node_ip = '192.168.2.20'
node_port = '30017'
url = f'mongodb://{user}:{password}@{node_ip}:{node_port}/'
client = pymongo.MongoClient(url, serverSelectionTimeoutMS = 2000)
client.server_info()

In [ ]:
print('Databases:', client.list_database_names())
db = client['test']
print('Collections:', db.list_collection_names())
collection = db['prod']
print('First Document:', str(collection.find_one())[:100])

# Get Data

In [ ]:
def get_data(datetime_start, datetime_end, supplier, model):
    assert isinstance(datetime_start, dt.datetime)
    assert isinstance(datetime_end, dt.datetime)
    
    query = {
        'metadata.request_datetime_utc': {
            '$gte': datetime_start,
            '$lt': datetime_end
        },
        'request.supplier': supplier,
        'request.model': model
    }
    
    data_list = []
    for x in collection.find(query):
        data_list.append(x)
    
    return data_list

# Plot Data

In [ ]:
def get_error_flags(data):
    assert isinstance(data, list)
    
    X_error, X_ok = [], []
    for x in data:
        request_datetime_utc = x['metadata']['request_datetime_utc']
        if x['result'] is None:
            X_error.append(request_datetime_utc)
        else:
            X_ok.append(request_datetime_utc)
    
    return X_error, X_ok

In [ ]:
def get_invalid_flags(data):
    assert isinstance(data, list)
    
    X_has_invalid, X_all_valid = [], []
    for x in data:
        request_datetime_utc = x['metadata']['request_datetime_utc']
        if x['result'] is not None:
            num_invalid = x['result']['products_invalid'] if 'products_invalid' in x['result'] else 0
            if num_invalid > 0:
                X_has_invalid.append(request_datetime_utc)
            else:
                X_all_valid.append(request_datetime_utc)  
    
    return X_has_invalid, X_all_valid

In [ ]:
def get_found_available_flags(data):
    assert isinstance(data, list)
    
    X_has_found, X_has_available = [], []

    for x in data:
        request_datetime_utc = x['metadata']['request_datetime_utc']
        if x['result'] is not None:
            num_found = x['result']['products_found']
            num_available = x['result']['products_available']

            if num_found > 0:
                X_has_found.append(request_datetime_utc)
            if num_available > 0:
                X_has_available.append(request_datetime_utc)
    
    return X_has_found, X_has_available

In [ ]:
def get_flags(data):
    X_error, X_ok = get_error_flags(data)
    X_has_invalid, X_all_valid = get_invalid_flags(data)
    X_has_found, X_has_available = get_found_available_flags(data)

    return {
        'X_error': X_error,
        'X_ok': X_ok,
        'X_has_invalid': X_has_invalid,
        'X_all_valid': X_all_valid,
        'X_has_found': X_has_found,
        'X_has_available': X_has_available,
    }

In [ ]:
def plot_flags(data):
    
    flags = get_flags(data)
    
    X_error = flags['X_error']
    X_ok = flags['X_ok']
    X_has_invalid = flags['X_has_invalid']
    X_all_valid = flags['X_all_valid']
    X_has_found = flags['X_has_found']
    X_has_available = flags['X_has_available']
    
    _, ax = plt.subplots(figsize=(40,0.5))

    ax.scatter(X_has_found, [0]*len(X_has_found), color='lightgray')
    ax.scatter(X_has_available, [0]*len(X_has_available), color='green')

    ax.scatter(X_all_valid, [-1]*len(X_all_valid), color='lightgray')
    ax.scatter(X_has_invalid, [-1]*len(X_has_invalid), color='orange')

    ax.scatter(X_ok, [-2]*len(X_ok), color='lightgray')
    ax.scatter(X_error, [-2]*len(X_error), color='red')

    ax.set_ylim((-2.5, 0.5))
    ax.set_xlim((datetime_start, datetime_utcnow))

    plt.show()

In [ ]:
def plot_found_available(data):
    X, Y_num_found, Y_num_invalid, Y_num_available = [], [], [], []

    for x in data:
        request_datetime_utc = x['metadata']['request_datetime_utc']
        if x['result'] is not None:
            num_found = x['result']['products_found']
            num_valid = x['result']['products_valid'] if 'products_valid' in x['result'] else num_found
            num_invalid = x['result']['products_invalid'] if 'products_invalid' in x['result'] else 0
            assert num_valid + num_invalid == num_found
            num_available = x['result']['products_available']

            X.append(request_datetime_utc)
            Y_num_found.append(num_found)
            Y_num_invalid.append(num_invalid)
            Y_num_available.append(num_available)
    
    _, ax = plt.subplots(figsize=(40,4))
    ax.scatter(X, Y_num_found, color='black', marker='_')
    ax.scatter(X, Y_num_invalid, color='orange', marker='_')
    ax.scatter(X, Y_num_available, color='green', marker='_')
    
    ax.set_xlim((datetime_start, datetime_utcnow))
    plt.show()

# Process Data

In [ ]:
hours=1.0
offset=0

datetime_utcnow = dt.datetime.utcnow() - dt.timedelta(hours=offset)
datetime_start = datetime_utcnow - dt.timedelta(hours=hours) - dt.timedelta(hours=offset)

In [ ]:
requests_list = [
    {'supplier': 'ebuyer', 'model': '3080'},
    {'supplier': 'ebuyer', 'model': '3070'},
    {'supplier': 'ebuyer', 'model': '3060ti'},
]

In [ ]:
data_list = []

for request in requests_list:
    data = get_data(datetime_start, datetime_utcnow, supplier=request['supplier'], model=request['model'])
    data_list.append(data)
    print(len(data))

In [ ]:
for i in range(len(requests_list)):
    print(requests_list[i])
    plot_flags(data_list[i])

In [ ]:
for i in range(len(requests_list)):
    print(requests_list[i])
    plot_found_available(data)